# Example: Set up an HDF5 plugin

In [1]:
from apstools.devices import AD_EpicsFileNameHDF5Plugin
from apstools.devices import AD_full_file_name_local
from apstools.devices import AD_prime_plugin2
from ophyd import EpicsSignalWithRBV
from ophyd.areadetector import ADComponent
from ophyd.areadetector import DetectorBase
from ophyd.areadetector import SingleTrigger
from ophyd.areadetector.plugins import ImagePlugin_V34 as ImagePlugin
from ophyd.areadetector.plugins import PvaPlugin_V34 as PvaPlugin
from ophyd.areadetector import SimDetectorCam
import pathlib
import time

In [2]:
IOC = "ad:"
IMAGE_DIR = "adsimdet/%Y/%m/%d"
AD_IOC_MOUNT_PATH = pathlib.Path("/tmp")
BLUESKY_MOUNT_PATH = pathlib.Path("/tmp/docker_ioc/iocad/tmp")

In [3]:
# MUST end with a `/`, pathlib will NOT provide it
WRITE_PATH_TEMPLATE = f"{AD_IOC_MOUNT_PATH / IMAGE_DIR}/"
READ_PATH_TEMPLATE = f"{BLUESKY_MOUNT_PATH / IMAGE_DIR}/"

In [4]:
class SimDetectorCam_R3_1_1(SimDetectorCam):
    """Revise SimDetectorCam for ADcore revisions."""
    pool_max_buffers = None
    offset = ADComponent(EpicsSignalWithRBV, "Offset")

In [5]:
class MySimDetector(SingleTrigger, DetectorBase):
    """ADSimDetector"""

    cam = ADComponent(SimDetectorCam_R3_1_1, "cam1:")
    hdf1 = ADComponent(
        AD_EpicsFileNameHDF5Plugin,
        "HDF1:",
        write_path_template=WRITE_PATH_TEMPLATE,
        read_path_template=READ_PATH_TEMPLATE,
    )
    # image = ADComponent(ImagePlugin, "image1:")
    # pva = ADComponent(PvaPlugin, "Pva1:")

In [6]:
adsimdet = MySimDetector(IOC, name="adsimdet")
adsimdet.wait_for_connection(timeout=15)

In [7]:
AD_prime_plugin2(adsimdet.hdf1)

In [8]:
# these settings are the caller's responsibility
adsimdet.hdf1.create_directory.put(-5)
adsimdet.hdf1.file_path.put(WRITE_PATH_TEMPLATE)  # path that the IOC will write the file
adsimdet.hdf1.file_template.put(f"%s%s_%4.4d.h5")
adsimdet.hdf1.file_name.put("my_test_file")
adsimdet.hdf1.auto_increment.put("Yes")
adsimdet.hdf1.auto_save.put("Yes")
adsimdet.hdf1.compression.put("zlib")
adsimdet.hdf1.zlevel.put(6)

In [9]:
adsimdet.read_attrs.append("hdf1")

In [10]:
adsimdet.stage()
adsimdet.trigger()
time.sleep(0.005)
while adsimdet.cam.acquire.get(use_monitor=False) not in (0, "Done"):
    time.sleep(0.005)
adsimdet.unstage()

In [11]:
print(f"{adsimdet.hdf1.full_file_name.get(use_monitor=False)=}")

adsimdet.hdf1.full_file_name.get(use_monitor=False)='/tmp/adsimdet/2022/07/06/my_test_file_0137.h5'


In [12]:
lfname = AD_full_file_name_local(adsimdet.hdf1)
print(f"{lfname.exists()=} {lfname=}")

lfname.exists()=True lfname=PosixPath('/tmp/docker_ioc/iocad/tmp/adsimdet/2022/07/06/my_test_file_0137.h5')
